<a href="https://colab.research.google.com/github/flatwaze/Learning-Algorithms-Implementations/blob/main/basic_fnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as f

import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd

from sklearn.model_selection import train_test_split

In [3]:
class Model_Iris(nn.Module):
  #Input layer
  def __init__(self, in_features=4, h1=8, h2=8, out_features=3):
    super().__init__()
    self.fc1 = nn.Linear(in_features, h1)
    self.fc2 = nn.Linear(h1, h2)
    self.out = nn.Linear(h2, out_features)

  def forward(self, x):
    x = f.relu(self.fc1(x))
    x = f.relu(self.fc2(x))
    x = self.out(x)
    return x

In [4]:
torch.manual_seed(100)
model = Model_Iris()

In [7]:
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
colnames = ['sepal.length', 'sepal.width', 'petal.length', 'petal.width',  'variety']
my_df = pd.read_csv(url, sep=',', names=colnames)

my_df['variety'] = my_df['variety'].replace('Iris-setosa', 0.0)
my_df['variety'] = my_df['variety'].replace('Iris-versicolor', 1.0)
my_df['variety'] = my_df['variety'].replace('Iris-virginica', 2.0)

my_df

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2.0
146,6.3,2.5,5.0,1.9,2.0
147,6.5,3.0,5.2,2.0,2.0
148,6.2,3.4,5.4,2.3,2.0


In [8]:
x = my_df.drop('variety', axis=1).values
y = my_df['variety'].values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
x_train, x_test = torch.FloatTensor(x_train), torch.FloatTensor(x_test)
y_train, y_test = torch.LongTensor(y_train), torch.LongTensor(y_test)


In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

epochs = 200
losses = []

for i in range(epochs):
  y_pred = model.forward(x_train)
  loss = criterion(y_pred, y_train) #predicted vs actual
  losses.append(loss.detach().numpy())

  if i%10 == 0:
    print(f'Epoch: {i} and loss: {loss}')

  #back prop.
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

Epoch: 0 and loss: 1.1285874843597412
Epoch: 10 and loss: 0.8888084888458252
Epoch: 20 and loss: 0.602279782295227
Epoch: 30 and loss: 0.3952573239803314
Epoch: 40 and loss: 0.23880812525749207
Epoch: 50 and loss: 0.12757927179336548
Epoch: 60 and loss: 0.07882557809352875
Epoch: 70 and loss: 0.06100289151072502
Epoch: 80 and loss: 0.05388832837343216
Epoch: 90 and loss: 0.05049600824713707
Epoch: 100 and loss: 0.04856621474027634
Epoch: 110 and loss: 0.04728967696428299
Epoch: 120 and loss: 0.046352557837963104
Epoch: 130 and loss: 0.04561396688222885
Epoch: 140 and loss: 0.04500555992126465
Epoch: 150 and loss: 0.044487837702035904
Epoch: 160 and loss: 0.044037509709596634
Epoch: 170 and loss: 0.043639324605464935
Epoch: 180 and loss: 0.04328298568725586
Epoch: 190 and loss: 0.04296109452843666


In [10]:
acc = 0
with torch.no_grad():
  y_eval = model.forward(x_test)
  loss = criterion(y_eval, y_test)
  for i in range (len(y_test)):
    if y_eval[i].argmax().item() == y_test[i]:
      acc += 1

print(loss)
print(acc/len(y_test))

tensor(0.0869)
0.9666666666666667


In [17]:
new_iris = torch.tensor([3, 2, 4, 2.2])
with torch.no_grad():
  y = model(new_iris)
  print(y.argmax().item())

2
